In [ ]:
%pip install --upgrade --quiet  langchain openai pypdf tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from langchain.chat_models import ChatAnyscale
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
anyscale_endpoint_token = userdata.get('ANYSCALE_TOKEN')

In [ ]:
llm = ChatAnyscale(
    anyscale_api_key=anyscale_endpoint_token,
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0,
)

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Using Langchain's PDF loader to extract text from a pdf file
loader = PyPDFLoader("/content/Marriage-Registration-Ordinance-No-19-of-1907-E.pdf")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Splitting to blocks of 1000 characters with a 200 overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# Creating the chunks
chunks = text_splitter.split_documents(data)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Defining the embedding model
embeddings = OpenAIEmbeddings(deployment="text-embedding-3-small",chunk_size = 1)

# Creating the Embeddings and storing in the Vector Database (Chromadb)
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# We use the Vector Store as Retriever
retriever = vectorstore.as_retriever()

from langchain.prompts import ChatPromptTemplate

template = """You are a legal assistant that answers questions about Sri Lankan Marraige Law. You must answer the question according to the context given to you. If the context does not contain the answer, you will say that you don't know.

Question: {question}

Context: {context}

Chat History:
{chat_history}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    k=1,
    memory_key='chat_history',
    return_messages=True,
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)

In [ ]:
from IPython.display import Markdown, display
def pp_markdown(content):
    display(Markdown(content))

In [ ]:
question = "What are the legal requirements for the solemnization of marriage in the presence of a registrar?"
result = qa({"question": question})
# Check the result of the query
pp_markdown(result['answer'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 To answer your question, the legal requirements for the solemnization of marriage in the presence of a registrar in Sri Lanka are as follows:

1. There must be no lawful impediment to the marriage.
2. The marriage must be solemnized in a registered place of worship or other authorized place.
3. The marriage must be solemnized by or in the presence of the minister of the building or a minister authorized by him.
4. The marriage must be solemnized between six o'clock in the morning and six o'clock in the afternoon.
5. The marriage must be solemnized in the presence of two or more credible witnesses.

If you have any further questions or need more information, please let me know.

In [ ]:
print(result['answer'])

 To answer your question, the legal requirements for the solemnization of marriage in the presence of a registrar in Sri Lanka are as follows:

1. There must be no lawful impediment to the marriage.
2. The marriage must be solemnized in a registered place of worship or other authorized place.
3. The marriage must be solemnized by or in the presence of the minister of the building or a minister authorized by him.
4. The marriage must be solemnized between six o'clock in the morning and six o'clock in the afternoon.
5. The marriage must be solemnized in the presence of two or more credible witnesses.

If you have any further questions or need more information, please let me know.
